In [7]:
import pandas as pd

with open('./query.txt', 'r') as f:
    list=f.readlines()
keyword=[]
for domain in list:
    a = domain.strip().split('.')[-2]
    if len(a)>2:
        keyword.append(a)
df = pd.DataFrame(keyword, columns=["keyword"])
df.to_csv('./keyword.csv', index=False)